In [1]:
!pip install beautifulsoup4

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [2]:
!pip install html5lib

Requirement not upgraded as not directly required: html5lib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: setuptools>=18.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)


In [3]:
!pip install requests

Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)


#### Reading the page

In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
soup = BeautifulSoup(source,'lxml')

#### Find the tbody part it contains the PostCode, Borough and Neighborhood table

In [7]:
tb=soup.find('tbody')


#### Loop through the table body, get every row via TR tag

In [23]:
import pandas as pd

df = pd.DataFrame(columns=('PostCode', 'Borough', 'Neighborhood'))
df.head()

for row in tb.find_all('tr'):
    try:
        fields = row.find_all('td')
        postcode = fields[0].text.replace('\n','')
        borough = fields[1].text.replace('\n','')
        neighborhood = fields[2].text.replace('\n','')
    
    
    
        if borough != 'Not assigned':
            if neighborhood == 'Not assigned':
                neighborhood = borough           
            
            
            df=df.append(pd.DataFrame([[postcode,borough,neighborhood]],columns=('PostCode', 'Borough', 'Neighborhood')))
        
    except Exception as e:
            yt_link = None      
    

In [24]:
df.head(10)


,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
0,M4A,North York,Victoria Village
0,M5A,Downtown Toronto,Harbourfront
0,M5A,Downtown Toronto,Regent Park
0,M6A,North York,Lawrence Heights
0,M6A,North York,Lawrence Manor
0,M7A,Queen's Park,Queen's Park
0,M9A,Etobicoke,Islington Avenue
0,M1B,Scarborough,Rouge
0,M1B,Scarborough,Malvern


In [25]:
df.shape


(211, 3)

#### Group by PostCode and Bourough and The Neighborhoood will be a list 

In [26]:
df2=df.groupby(['PostCode','Borough'])['Neighborhood'].apply(list)
df2.head(10)


PostCode  Borough    
M1B       Scarborough                                     [Rouge, Malvern]
M1C       Scarborough             [Highland Creek, Rouge Hill, Port Union]
M1E       Scarborough                  [Guildwood, Morningside, West Hill]
M1G       Scarborough                                             [Woburn]
M1H       Scarborough                                          [Cedarbrae]
M1J       Scarborough                                [Scarborough Village]
M1K       Scarborough        [East Birchmount Park, Ionview, Kennedy Park]
M1L       Scarborough                    [Clairlea, Golden Mile, Oakridge]
M1M       Scarborough    [Cliffcrest, Cliffside, Scarborough Village West]
M1N       Scarborough                        [Birch Cliff, Cliffside West]
Name: Neighborhood, dtype: object

In [27]:
df2.shape

(103,)

In [28]:
df2 = df2.to_frame().reset_index()

In [29]:
!wget -O location.csv https://cocl.us/Geospatial_data

--2019-04-29 22:11:59--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-29 22:12:01--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-29 22:12:02--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-2

In [30]:
loc_df = pd.read_csv('location.csv')
loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
result = pd.merge(df2,
                 loc_df,
                 left_on='PostCode',
                 right_on='Postal Code')
result.head()

,PostCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",M1B,43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",M1C,43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",M1E,43.763573,-79.188711
3,M1G,Scarborough,[Woburn],M1G,43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],M1H,43.773136,-79.239476


In [32]:
del result['Postal Code']

In [33]:
result.head(10)

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848


In [34]:
result.shape

(103, 5)

#### Check the number of Borough and Neighborhood

In [37]:
print('The dataframe has  {} boroughs and {} neighborhoods.'.format(
    len(result['Borough'].unique()),
    result.shape[0]
     )
)

The dataframe has  11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [40]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.31 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.78 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  41.65 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  49.59 MB/s
Libraries imported.


In [44]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="torronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [47]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Now only work with boroughs that contain the word Toronto 

In [63]:
toronto_data = result[result['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,index,PostCode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,[The Beaches],43.676357,-79.293031
1,41,M4K,East Toronto,"[The Danforth West, Riverdale]",43.679557,-79.352188
2,42,M4L,East Toronto,"[The Beaches West, India Bazaar]",43.668999,-79.315572
3,43,M4M,East Toronto,[Studio District],43.659526,-79.340923
4,44,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790


#### Create a map

In [64]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_boroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Fooursquare API to explore the neighborhoods and segment them

#### Define Fourspace Credentials and Version

In [62]:
CLIENT_ID = 'BZNRGRPJZMDAF2REEYRCU0NF2G1KISBPSW14FIXV0WZNDNZB' # your Foursquare ID
CLIENT_SECRET = 'VZ2T2NYEQZ2H3YKLP2CA0ODHYBXNTPPFZTHVVMD3AXHHYRUQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BZNRGRPJZMDAF2REEYRCU0NF2G1KISBPSW14FIXV0WZNDNZB
CLIENT_SECRET:VZ2T2NYEQZ2H3YKLP2CA0ODHYBXNTPPFZTHVVMD3AXHHYRUQ


#### Let's explore the first neighborhood in our dataframe.

In [65]:
toronto_data.loc[0, 'Neighborhood']

['The Beaches']

In [66]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['The Beaches'] are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venus that are in The Beaches within a radius of 500 meters

In [67]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=BZNRGRPJZMDAF2REEYRCU0NF2G1KISBPSW14FIXV0WZNDNZB&client_secret=VZ2T2NYEQZ2H3YKLP2CA0ODHYBXNTPPFZTHVVMD3AXHHYRUQ&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [68]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc77e2d4c1f674392e5ddee'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c062f964a52011f820e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '50aa9e744b90af0d42d5de0e',
         'name': 'Health Food Store',
         'pluralName': 'Health Food Stores',
         'primary': True,
         'shortName': 'Health Food Store'}],
       'id': '4ad4c062f964a52011f820e3',
       'location': {'address': '125 Southwood Dr',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 471,
        'formattedAddress': ['125 Southwood Dr',
         'Toronto ON M4E 0B8',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
      

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [69]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [70]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Glen Manor Ravine,Trail,43.676821,-79.293942
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [71]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

['The Beaches']
['The Danforth West', 'Riverdale']
['The Beaches West', 'India Bazaar']
['Studio District']
['Lawrence Park']
['Davisville North']
['North Toronto West']
['Davisville']
['Moore Park', 'Summerhill East']
['Deer Park', 'Forest Hill SE', 'Rathnelly', 'South Hill', 'Summerhill West']
['Rosedale']
['Cabbagetown', 'St. James Town']
['Church and Wellesley']
['Harbourfront', 'Regent Park']
['Ryerson', 'Garden District']
['St. James Town']
['Berczy Park']
['Central Bay Street']
['Adelaide', 'King', 'Richmond']
['Harbourfront East', 'Toronto Islands', 'Union Station']
['Design Exchange', 'Toronto Dominion Centre']
['Commerce Court', 'Victoria Hotel']
['Roselawn']
['Forest Hill North', 'Forest Hill West']
['The Annex', 'North Midtown', 'Yorkville']
['Harbord', 'University of Toronto']
['Chinatown', 'Grange Park', 'Kensington Market']
['CN Tower', 'Bathurst Quay', 'Island airport', 'Harbourfront West', 'King and Spadina', 'Railway Lands', 'South Niagara']
['Stn A PO Boxes 25 The Es

#### Let's check the size of the resulting dataframe

In [80]:
print(toronto_venues.shape)
toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,[The Beaches],43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,[The Beaches],43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,[The Beaches],43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,[The Beaches],43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
4,[The Beaches],43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
